In [10]:
import numpy as np
import pandas as psd

In [2]:
def get_gender_dict(path):
    with open(path, 'r',  encoding="ISO-8859-1") as f:
        contents = f.read()
    split_content = contents.split('\n')
    split_content = split_content[0:-1]
    split_content_list=[]
    for i in split_content:
        split_content_list.append(i.split(",")[1:])
    split_content_list=split_content_list[1:]
    
    gender_dict = {}
    for split_data in split_content_list:
        gender_dict[split_data[0]] = split_data[1]
    f.close()    
    return gender_dict

In [3]:
path_gender_profile = "./data/profile_gender.csv"
user_gender_dictionary = get_gender_dict(path_gender_profile)
male_list = []
female_list = []
for key in user_gender_dictionary.keys():
    if user_gender_dictionary[key] == '1':
        male_list.append(1)
    else:
        female_list.append(1)
N_males = len(male_list) 
N_females = len(female_list)

In [4]:
print(N_males, N_females)

775142 395731


In [5]:
def fairness_score(I_males, I_females, N_males, N_females):
    male_ratio = I_males/N_males
    female_ratio = I_females/N_females
    ratio = [male_ratio, female_ratio]
    prod = 1
    for x in ratio:
        prod *= x
    return prod/(np.max(ratio)**len(ratio))

In [6]:
f = open("./data/train_cascades.txt","r")

In [7]:
influencer_cascades_dict = {}

In [8]:
for line in f:
    initiators = []
    cascade = line.replace("\n","").split(";")
    cascade_nodes = list(map(lambda x:  x.split(" ")[0],cascade[1:]))
    op_id = cascade_nodes[0]
    if op_id not in influencer_cascades_dict:
        influencer_cascades_dict[op_id] = cascade_nodes
    else:
        influencer_cascades_dict[op_id] = influencer_cascades_dict[op_id] + cascade_nodes


In [9]:
print("Evaluating fairnsess score of each influencer in train_cascades")
influencer_fairness = open("./data/influencer_fairness.txt","w")
##########################CHANDU: EXPERIMENT ##############################    
for k in influencer_cascades_dict.keys():
    cascade_unique = set(influencer_cascades_dict[k])
    male_list_cascade = []
    female_list_cascade = []
    for cascade_node in cascade_unique:
        gender = user_gender_dictionary[cascade_node]
        if gender == '1':
            male_list_cascade.append(1)
        else:
            female_list_cascade.append(1)

    fainess_score =  round(fairness_score(len(male_list_cascade), len(female_list_cascade), N_males, N_females), 4)
    influencer_fairness.write(str(k)+","+ str(fainess_score) + "\n")    
        
influencer_fairness.close()

Evaluating fairnsess score of each influencer in train_cascades


In [10]:
influencer_fairness_read = open("./data/influencer_fairness.txt","r")
gender_dict_read = {}
for line in influencer_fairness_read:
    data = line.replace("\n","").split(",")
    gender_dict_read[data[0]] = data[1]
   